### # I wanted to create a chatbot from [my old livejournal entries from around 2001 onward]("https://www.livejournal.com/userinfo.bml?user=laleche"). I downloaded the content as raw CSV files and using R I first aggregated the text using rbind() to create one csv.  This is a small dataset overall but I wanted to just get started and hone the model csv later. <br><br>
### I loaded the livejournal blog entries and began to code.

In [105]:
import pandas as pd

file_path = "/Users/jennyskytta/Downloads/lj2001.csv"
df = pd.read_csv(file_path)
df.columns

Index(['itemid', 'eventtime', 'logtime', 'subject', 'event', 'security',
       'allowmask', 'current_music', 'current_mood'],
      dtype='object')

### most of the columns downloaded arent needed, so I dropped everything except the subject and event which has the actual blog text. 

In [106]:
# Assuming you have a DataFrame named 'df' with the mentioned columns
columns_to_drop = ['itemid', 'eventtime', 'logtime', 'security', 'allowmask', 'current_music', 'current_mood']

# Drop the specified columns
df = df.drop(columns=columns_to_drop)

# Now, 'df' contains only the 'event' column
print(df)

                                      subject  \
0                           Baby's First Step   
1                                     Le Show   
2    Practice is a bust..Thank You Christians   
3                            Ring a ding ding   
4                           Bugs are a biting   
..                                        ...   
97                             I think not...   
98                               Neurolux 101   
99                                  Car Talk`   
100                                   amusing   
101           I need a vacation from thinking   

                                                 event  
0    wooo hooo!!!!   Okay..I'm really just murderin...  
1    Yay!!! The monotany of daily nothingness was b...  
2    Our practice never happened...because once I a...  
3    Blase today, I worked mucho hours.  Nothing ex...  
4    Yikes...today was horrid.  Awakened from my sl...  
..                                                 ...  
97   <TABLE>

### From this sample, you can see that early 20s version of me was very crass. 

In [107]:
# Pad the 'subject' column with NaN to make it the same length as the 'event' column
max_length = max(len(df['subject']), len(df['event']))
df['subject'] = df['subject'].reindex(range(max_length)).fillna(method='ffill')

# Pad the 'event' column with NaN to make it the same length as the 'subject' column
df['event'] = df['event'].reindex(range(max_length)).fillna(method='ffill')

# Now both columns have the same length
df.shape

(102, 2)

In [109]:
import pandas as pd
import csv

# Create a new DataFrame to hold the questions and answers
qa_df = pd.DataFrame(columns=['Question', 'Answer'])

# Iterate over each row in the original CSV file
for index, row in df.iterrows():
    subject = row['subject']
    event = row['event']
    
    # Generate potential questions and answers based on the subject and event
    # Replace this with your own code to generate relevant questions and answers
    questions = [
        'What is '+subject+'?',
        'Can you explain '+subject+'?',
        'What are the key points of '+subject+'?',
        'How does '+subject+' relate to '+event+'?',
        'What are some interesting facts about '+subject+'?',
        'Who are the experts on '+subject+'?',
        'What are the benefits of learning about '+subject+'?',
        'What are the challenges of learning about '+subject+'?',
        'What is the history of '+subject+'?',
        'What are the future trends of '+subject+'?'
    ]
    answers = ['Sorry, no answer found for '+subject+'.']*10

    # Add the generated questions and answers to the new DataFrame
    qa_df = qa_df.append(pd.DataFrame({'Question': questions, 'Answer': answers}), ignore_index=True)

# Remove duplicates from the resulting dataframe
qa_df.drop_duplicates(inplace=True)

# Save the new DataFrame as a CSV file
qa_df.to_csv('chatbot_qa.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)


/var/folders/rc/ylf7hxm13x365r22_1xp_ybc0000gn/T/ipykernel_5370/2107150349.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qa_df = qa_df.append(pd.DataFrame({'Question': questions, 'Answer': answers}), ignore_index=True)


In [110]:
qa_df

,Question,Answer
0,What is Baby's First Step?,"Sorry, no answer found for Baby's First Step."
1,Can you explain Baby's First Step?,"Sorry, no answer found for Baby's First Step."
2,What are the key points of Baby's First Step?,"Sorry, no answer found for Baby's First Step."
3,How does Baby's First Step relate to wooo hooo...,"Sorry, no answer found for Baby's First Step."
4,What are some interesting facts about Baby's F...,"Sorry, no answer found for Baby's First Step."
...,...,...
1015,Who are the experts on I need a vacation from ...,"Sorry, no answer found for I need a vacation f..."
1016,What are the benefits of learning about I need...,"Sorry, no answer found for I need a vacation f..."
1017,What are the challenges of learning about I ne...,"Sorry, no answer found for I need a vacation f..."
1018,What is the history of I need a vacation from ...,"Sorry, no answer found for I need a vacation f..."


In [111]:
def preprocess(text):
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text.strip()

In [119]:
import re

qa_df['Question'] = qa_df['Question'].apply(preprocess)
qa_df['Answer'] = qa_df['Answer'].apply(preprocess)

In [120]:
def find_closest_match(user_input):
    user_input = preprocess(user_input)
    distances = qa_df['Question'].apply(lambda x: abs(len(x) - len(user_input)))
    index = distances.idxmin()
    return qa_df.loc[index, 'Answer']

## Step 8: Create a chat loop

In [ ]:
print("Welcome to the Jenny Chatbot! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = find_closest_match(user_input)
    print("Chatbot: ", response)

Welcome to the Jenny Chatbot! Type 'quit' to exit.
You: hello
Chatbot:  sorry no answer found for tim
You: what is your name?
Chatbot:  sorry no answer found for dirty pop
You: what is dirty pop
Chatbot:  sorry no answer found for dirty pop
You: I'm confused
Chatbot:  sorry no answer found for augh
You: this is weird
Chatbot:  sorry no answer found for augh
You: I'm so confused
Chatbot:  sorry no answer found for le show
You: what is le show?
Chatbot:  sorry no answer found for le show


### This really requires that I have a much richer model of data but for a rough bot, she works 